In [11]:
import pandas as pd
import concurrent.futures
from skyfield.api import load, Topos
import numpy as np

def calculate_sun_position(row):
    t = ts.utc(year=row['y'], month=row['m'], day=row['d'], hour=row['h'], minute=row['mm'], second=row['s'])
    ast_position = observer.at(t).observe(asteroid)
    x, y, z = ast_position.apparent().position.au
    return [row['time'], x, y, z]

df = pd.read_csv('D:\Earthquake\EPH\DATE_1950_2024.csv')
df['time'] = pd.to_datetime(df['time'])
df['y'] = df['time'].dt.year
df['m'] = df['time'].dt.month
df['d'] = df['time'].dt.day
df['h'] = df['time'].dt.hour
df['mm'] = df['time'].dt.minute
df['s'] = df['time'].dt.second

print(df.head())

eph = load('de421.bsp')
earth = eph['earth']
observer = earth + Topos(latitude_degrees=0, longitude_degrees=0)
asteroid = eph['Neptune BARYCENTER']
ts = load.timescale()

def process_chunk(chunk):
    sun_pos_chunk = chunk.apply(calculate_sun_position, axis=1)
    return sun_pos_chunk.values.tolist()

num_parts = 8
chunks = np.array_split(df, num_parts)

with concurrent.futures.ThreadPoolExecutor() as executor:
    results = list(executor.map(process_chunk, chunks))

sun_pos = [item for sublist in results for item in sublist]
dt = pd.DataFrame(sun_pos, columns=['time', 'x', 'y', 'z'])
dt.to_csv('Neptune.csv', index=False)


        time     y  m  d  h  mm  s
0 1950-01-01  1950  1  1  0   0  0
1 1950-01-02  1950  1  2  0   0  0
2 1950-01-03  1950  1  3  0   0  0
3 1950-01-04  1950  1  4  0   0  0
4 1950-01-05  1950  1  5  0   0  0


In [13]:
from skyfield.api import load, Topos
eph = load('de421.bsp')
print(eph)

SPICE kernel file 'de421.bsp' has 15 segments
  JD 2414864.50 - JD 2471184.50  (1899-07-28 through 2053-10-08)
      0 -> 1    SOLAR SYSTEM BARYCENTER -> MERCURY BARYCENTER
      0 -> 2    SOLAR SYSTEM BARYCENTER -> VENUS BARYCENTER
      0 -> 3    SOLAR SYSTEM BARYCENTER -> EARTH BARYCENTER
      0 -> 4    SOLAR SYSTEM BARYCENTER -> MARS BARYCENTER
      0 -> 5    SOLAR SYSTEM BARYCENTER -> JUPITER BARYCENTER
      0 -> 6    SOLAR SYSTEM BARYCENTER -> SATURN BARYCENTER
      0 -> 7    SOLAR SYSTEM BARYCENTER -> URANUS BARYCENTER
      0 -> 8    SOLAR SYSTEM BARYCENTER -> NEPTUNE BARYCENTER
      0 -> 9    SOLAR SYSTEM BARYCENTER -> PLUTO BARYCENTER
      0 -> 10   SOLAR SYSTEM BARYCENTER -> SUN
      3 -> 301  EARTH BARYCENTER -> MOON
      3 -> 399  EARTH BARYCENTER -> EARTH
      1 -> 199  MERCURY BARYCENTER -> MERCURY
      2 -> 299  VENUS BARYCENTER -> VENUS
      4 -> 499  MARS BARYCENTER -> MARS
